In [4]:
import sys
import scipy as sc
from lmfit import Minimizer, minimize,Parameters, report_fit
from numpy import exp, log
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from model_fittings import *
import subprocess as sp
from subprocess import os
import time

In [5]:
k = 8.617e-5
d = pd.DataFrame(pd.read_csv("../Data/ThermRespData.csv"))
d = d[d['OriginalTraitValue'] > 0]
d['LoggedOriginalTraitValue'] = np.log(d['OriginalTraitValue'])
d = d[["ID", "ConTemp", "OriginalTraitValue",
        "LoggedOriginalTraitValue", "OrignalTraitName",  "OriginalTraitUnit"]]
d["ConTemp_Kelvin"] = d["ConTemp"] + 273.15
d["Transferred_ConTemp"] = 1/k*d['ConTemp_Kelvin']+1/283.15*k
d["Transferred_kT"] = 1/k*d["ConTemp_Kelvin"]

In [7]:
id = 3
sub = d.loc[d["ID"] == id]; sub
# B0 = sub.loc[sub["ConTemp_Kelvin"] == np.min(sub["ConTemp_Kelvin"]),"OriginalTraitValue"]; B0
# ["LoggedOriginalTraitValue"]

,ID,ConTemp,OriginalTraitValue,LoggedOriginalTraitValue,OrignalTraitName,OriginalTraitUnit,ConTemp_Kelvin,Transferred_ConTemp,Transferred_kT
10,3,19.110678,1.638719,0.493915,change in o2 (photosynthesis),mg o2 (10 minutes^-1),292.260678,3.391676e+06,3.391676e+06
11,3,19.081886,1.699475,0.530320,change in o2 (photosynthesis),mg o2 (10 minutes^-1),292.231886,3.391341e+06,3.391341e+06
12,3,21.594273,1.811358,0.594077,change in o2 (photosynthesis),mg o2 (10 minutes^-1),294.744273,3.420498e+06,3.420498e+06
13,3,24.553116,1.956636,0.671227,change in o2 (photosynthesis),mg o2 (10 minutes^-1),297.703116,3.454835e+06,3.454835e+06
14,3,24.553661,1.917963,0.651264,change in o2 (photosynthesis),mg o2 (10 minutes^-1),297.703661,3.454841e+06,3.454841e+06
15,3,25.530449,1.979275,0.682730,change in o2 (photosynthesis),mg o2 (10 minutes^-1),298.680449,3.466177e+06,3.466177e+06
16,3,25.533960,1.730663,0.548505,change in o2 (photosynthesis),mg o2 (10 minutes^-1),298.683960,3.466217e+06,3.466217e+06
17,3,28.569110,2.406358,0.878114,change in o2 (photosynthesis),mg o2 (10 minutes^-1),301.719110,3.501440e+06,3.501440e+06
18,3,31.565092,1.899738,0.641716,change in o2 (photosynthesis),mg o2 (10 minutes^-1),304.715092,3.536209e+06,3.536209e+06
19,3,31.566029,1.833441,0.606195,change in o2 (photosynthesis),mg o2 (10 minutes^-1),304.716029,3.536219e+06,3.536219e+06


In [8]:
p = Parameters()
p.add("B0", value=0.1, min=0.00001, max=40)
p.add("E", value=0.5, min=0.000001, max=30)
p.add("Th", value=318, min=273.15 - 10, max=273.15 + 180)
p.add("Eh", value=30, min=0.000001, max=50)

In [9]:
T = np.asarray(sub["ConTemp_Kelvin"])  # Temperature in kelvin
B = np.asarray(sub["OriginalTraitValue"])  # Unlogged Trait value
B_log = np.asarray(sub["LoggedOriginalTraitValue"])  # Logged Trait value
output = minimize(resi_sch_log, p, args=(T, B_log))

In [14]:
output.params

name,value,initial value,min,max,vary
B0,1.53289823,0.1,1.0000e-05,40.0000000,True
E,0.09762728,0.5,1.0000e-06,30.0000000,True
Th,318.000000,318,263.150000,453.150000,True
Eh,30.0000000,30,1.0000e-06,50.0000000,True


In [19]:
# Making needed subdirectories in Results directory
print(os.system("mkdir ../Results/Figures"))
print(os.system("mkdir ../Results/Fitting_info"))

0
0


In [20]:
start = time.time()
Arrhenius_after = pd.DataFrame(data = None)
Arrhenius_before = pd.DataFrame(data = None)
quadratic = pd.DataFrame(data = None)
cubic = pd.DataFrame(data = None)
Briere = pd.DataFrame(data=None)
Schoolfield = pd.DataFrame(data=None)
for id in d["ID"].unique():
    if len(d[d["ID"] == id]) > 4: 
        print("Fitting Linear Modle of ID %d" % id)
        quad_info, cubic_info, fit_quad, fit_cubic = linear_models(id, d, only_return_info=False)
        quadratic = quadratic.append(quad_info)
        cubic = cubic.append(cubic_info)
        
        print("Fitting Briere Modle of ID %d" % id)
        briere_info, p_bri = Briere_model(
            id, d, min_round=140, max_round=180, only_return_info=False)
        Briere = Briere.append(briere_info)
        
        print("Fitting Simplified Schoolfield Modle of ID %d" % id)
        school_info, p_sch, arr_before, arr_after = Schoolfield_model(
            id, d, min_round=200, max_round=220, only_return_info=False)
        Schoolfield = Schoolfield.append(school_info)
        Arrhenius_before = Arrhenius_before.append(arr_before)
        Arrhenius_after = Arrhenius_after.append(arr_after)
        
        # Save pictures
        models_visualisation(id, d, (fit_quad, fit_cubic, p_bri, p_sch))
        plt.savefig("../Results/Figures/TPC_fitting%d.pdf" % id)
        plt.close()
print("Fitting process finished, taking %f minutes to run." % ((time.time() - start)/60))

Fitting Linear Modle of ID 1
Fitting Briere Modle of ID 1
Fitting Simplified Schoolfield Modle of ID 1
Fitting Linear Modle of ID 2
Fitting Briere Modle of ID 2
Fitting Simplified Schoolfield Modle of ID 2
Fitting Linear Modle of ID 3
Fitting Briere Modle of ID 3
Fitting Simplified Schoolfield Modle of ID 3
Fitting Linear Modle of ID 4
Fitting Briere Modle of ID 4
Fitting Simplified Schoolfield Modle of ID 4
Fitting Linear Modle of ID 5
Fitting Briere Modle of ID 5
Fitting Simplified Schoolfield Modle of ID 5
Fitting Linear Modle of ID 6
Fitting Briere Modle of ID 6
Fitting Simplified Schoolfield Modle of ID 6
Fitting Linear Modle of ID 7
Fitting Briere Modle of ID 7
Fitting Simplified Schoolfield Modle of ID 7
Fitting Linear Modle of ID 8
Fitting Briere Modle of ID 8
Fitting Simplified Schoolfield Modle of ID 8
Fitting Linear Modle of ID 9
Fitting Briere Modle of ID 9
Fitting Simplified Schoolfield Modle of ID 9
Fitting Linear Modle of ID 10
Fitting Briere Modle of ID 10
Fitting Simpl

In [21]:
print("-"*80, '\n')
quadratic.to_csv("../Results/Fitting_info/quad_info.csv", sep=",", index=False)
cubic.to_csv("../Results/Fitting_info/cubic_info.csv",
                sep=",", index=False)
Briere.to_csv("../Results/Fitting_info/Briere_info.csv",
                sep=",", index=False)
Schoolfield.to_csv(
    "../Results/Fitting_info/Schoolfield_info.csv", sep=",", index=False)
Arrhenius_before.to_csv(
    "../Results/Fitting_info/Arrhenius_before_info.csv")
Arrhenius_after.to_csv("../Results/Fitting_info/Arrhenius_after_info.csv")
print("Finished! The info of fitted models has been saved into Results directory." )

-------------------------------------------------------------------------------- 

Finished! The info of fitted models has been saved into Results directory.


In [16]:
# Run python and R scripts
print(os.system("Rscript model_analysis.R"))

0


In [ ]:
def Arrhenius_model(id, data):
    # k = 8.617e-5
    sub = data[data["ID"] == id]
    # T = np.asarray(data["ConTemp"][data['ID'] == id])  # Temperature in kelvin
    # B_log = np.asarray(data["LoggedOriginalTraitValue"][data['ID'] == id])  # Unlogged Trait value

    # T_max = sub["Transferred_ConTemp"][sub["LoggedOriginalTraitValue"] == np.max(sub["LoggedOriginalTraitValue"])]

    # if len(T_max) >= 2:
    #     T_max = np.mean(T_max)

    # B_log_max = sub["LoggedOriginalTraitValue"][sub["Transferred_ConTemp"] == np.min(
    #     sub["Transferred_ConTemp"])]
    # if len(B_log_max) >= 2:
    #     T_max = np.mean(B_log_max)

    B_log = np.asarray(sub["LoggedOriginalTraitValue"])
    Trans_T = np.asarray(sub["Transferred_ConTemp"])
    # logB_bef = np.asarray(sub["LoggedOriginalTraitValue"]
    #                       [sub["Transferred_ConTemp"] <= float(T_max)])
    # logB_af = np.asarray(sub["LoggedOriginalTraitValue"]
    #                      [sub["Transferred_ConTemp"] >= float(T_max)])
    # transT_bef = np.asarray(
    #     sub["Transferred_ConTemp"][sub["Transferred_ConTemp"] <= float(T_max)])  # Transferred Temperature
    # transT_af = np.asarray(
    #     sub["Transferred_ConTemp"][sub["Transferred_ConTemp"] >= float(T_max)])

    fit_arr = np.polyfit(Trans_T, B_log, 1)

    # Get fitting info of Arrhenius model of dataset BEFORE deactivation
    # if len(logB_bef) > 4 and len(logB_af) > 4:
    #     fitArr_bef = np.polyfit(transT_bef, logB_bef, 1)
    #     resi_bef = resi(fitArr_bef, transT_bef, logB_bef)

    #     rss_bef = sum(resi_bef ** 2)
    #     tss_bef = sum((logB_bef - np.mean(logB_bef)) ** 2)
    #     rsq_bef = 1 - (rss_bef/tss_bef)

    #     aic_bef = get_AIC(resi_bef, logB_bef, fitArr_bef)
    #     bic_bef = get_BIC(resi_bef, logB_bef, fitArr_bef)
    #     #aicc = aic + (2 * k * (k + 1)) / (len(T) - k - 1)
    #     aicc_bef = aic_bef + (2 * 2 * (2 + 1)) / ((len(T) - 2 - 1))

    #     results_all_before = {
    #         "id": [id],
    #         "n": len(T),
    #         "B0_log_before_deactivation": fitArr_bef[0],
    #         "E_before_deactivation": fitArr_bef[1],
    #         "Rsq_before_deactivation": rsq_bef,
    #         "aic_before_deactivation": aic_bef,
    #         "bic_before_deactivation": bic_bef,
    #         "aicc_before_deactivation": aicc_bef
    #     }

    # else:
    #     results_all_before = {
    #         "id": [id],
    #         "n": len(T),
    #         "B0_log_before_deactivation": B0_all,
    #         "E_before_deactivation": 0.5,
    #         "Rsq_before_deactivation": np.NaN,
    #         "aic_before_deactivation": np.NaN,
    #         "bic_before_deactivation": np.NaN,
    #         "aicc_before_deactivation": np.NaN
    #     }

    # Get fitting info of Arrhenius model of dataset AFTER deactivation
    # if len(logB_af) > 4:
    #     fitArr_af = np.polyfit(transT_af, logB_af, 1)
    #     resi_af = resi(fitArr_af, transT_af, logB_af)

    #     rss_af = sum(resi_af ** 2)
    #     tss_af = sum((logB_af - np.mean(logB_af)) ** 2)
    #     rsq_af = 1 - (rss_af/tss_af)

    #     aic_af = get_AIC(resi_af, logB_af, fitArr_af)
    #     bic_af = get_BIC(resi_af, logB_af, fitArr_af)
    #     aicc_af = aic_af + 2 * 2 * ((2 + 1)/(len(T) - 2 - 1))

    #     results_all_after = {
    #         "id": [id],
    #         "n": len(T),
    #         "B0_log_after_deactivation": fitArr_af[0],
    #         "E_after_deactivation": fitArr_af[1],
    #         "Rsq_after_deactivation": rsq_af,
    #         "aic_after_deactivation": aic_af,
    #         "bic_after_deactivation": bic_af,
    #         "aicc_after_deactivation": aicc_af
    #     }
    # else:
    #     results_all_after = {
    #         "id": [id],
    #         "n": len(T),
    #         "B0_log_after_deactivation": B0_all,
    #         "E_after_deactivation": 0.5,
    #         "Rsq_after_deactivation": np.NaN,
    #         "aic_after_deactivation": np.NaN,
    #         "bic_after_deactivation": np.NaN,
    #         "aicc_after_deactivation": np.NaN
    #     }
    if np.isnan(fit_arr[1]):
        return 10, 30
    else:
        return abs(fit_arr[1]), abs(fit_arr)[1] * 10